In [ ]:
!pip install langchain-teddynote markdownify pymupdf

In [2]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("Jimin-Parser-Test")

LangSmith 추적을 시작합니다.
[프로젝트명]
Jimin-Parser-Test


In [3]:
import os
import pymupdf
from glob import glob
import json
import requests
from PIL import Image

In [4]:
SAMPlE = "data/sample-report.pdf"

## layout parsing algorithm

레이아웃에서 가져온 이미지를 벡터디비에 넣고, 이미지를 다시 우리한테 꺼내서 보여줘야할 줄 알아야함. layout parser는 이 영역의 이미지가 존재한다를 알려주는거지, 이 이미지를 따로 따서 저장하지는 않음. 

- table도 이미지 캡처떠서 멀티 모달에 넣어서 그거에 마크다운 변환해서 넣어주면 성능이 좋아질것 같음. 


### 1. PDF 파일 분할

#### 왜 분할 할까?
- 100페이지 이상의 PDF 파일을 한 번에 처리할 경우 Upstage layout parsing API 오류 발생
- 처리 효율성 및 속도 개선
- 단일 페이지 처리는 api 전체 처리 시간을 증가시킴 

#### 일케 해봅세
- PDF 파일을 적절한 크기의 세그먼트로 분할하여 처리
- 분할된 세그먼트를 순차적으로 API에 전송하여 처리 효율성 향상

In [14]:
filepath = SAMPlE
batch_size = 10

input_pdf = pymupdf.open(filepath)
num_pages = len(input_pdf)
print(f"총 페이지 수: {num_pages}")


ret_split_files= []

# PDF 분할
for start_page in range(0,num_pages,batch_size):
    end_page = min(start_page + batch_size, num_pages) -1
    print(f"분할: {start_page} ~ {end_page}")
    
    
    # 분할된 PDF를 저장
    input_file_basename = os.path.splitext(filepath)[0]
    output_file = f"{input_file_basename}_{start_page}_{end_page}.pdf"
    print(f"분할 PDF 저장: {output_file}")
    with pymupdf.open() as output_pdf:
        output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
        output_pdf.save(output_file)
        ret_split_files.append(output_file)
        print(f"{output_file} ret에 추가 ")
        
input_pdf.close()

총 페이지 수: 22
분할: 0 ~ 9
분할 PDF 저장: data/sample-report_0_9.pdf
data/sample-report_0_9.pdf ret에 추가 
분할: 10 ~ 19
분할 PDF 저장: data/sample-report_10_19.pdf
data/sample-report_10_19.pdf ret에 추가 
분할: 20 ~ 21
분할 PDF 저장: data/sample-report_20_21.pdf
data/sample-report_20_21.pdf ret에 추가 


In [15]:
print(ret_split_files)

['data/sample-report_0_9.pdf', 'data/sample-report_10_19.pdf', 'data/sample-report_20_21.pdf']
